In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv("datasets/train.csv")

In [3]:
train_df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [4]:
train_df.columns = [column.lower() for column in train_df.columns]
train_df.columns = [column.replace(' ','_') for column in train_df.columns]
train_df.head()

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


### Neighborhood Dataframe

In [5]:
stonebr_df = train_df[(train_df['neighborhood'] == 'StoneBr')]
stonebr_nulls = stonebr_df.isna().sum().sort_values(ascending=False)
stonebr_nulls[stonebr_nulls > 0]

alley           38
misc_feature    38
fence           38
pool_qc         38
fireplace_qu     9
lot_frontage     4
dtype: int64

In [6]:
#drop empty features == ['Alley', 'Misc Feature', 'Fence', 'Pool QC']
stonebr_df = stonebr_df.drop(columns = ['alley', 'misc_feature', 'fence', 'pool_qc'])

In [7]:
#NaN values corresponde to no value, replace with 0 ['Fireplace Qu', 'Lot Frontage']
stonebr_df = stonebr_df.dropna(0)

### Test Train Split

In [8]:
X = stonebr_df.drop(columns = 'saleprice')
y = stonebr_df[['saleprice','id']]

### Split DataFrame Based on Type of Feature

In [9]:
#split the data based on type to effectivly explore,impute,and encode if needed. Drop target featurer

#categorical features
numeric_train_df = X.select_dtypes(include = ['int','float']).sort_index()

#categorical features
categorical_train_df =  X.select_dtypes(include = ['object']).sort_index()

# Numeric Features

In [10]:
#remove ID and PID, put into own DF
id_train_df = numeric_train_df[['id','pid']]

numeric_train_df = numeric_train_df.drop(columns = ['id','pid'])

In [11]:
numeric_null = numeric_train_df.isna().sum().sort_values(ascending = False)
numeric_null[numeric_null > 0]

Series([], dtype: int64)

#### Numeric categories that could mean 0 for NA


## Standard Scalar

In [12]:
ss = StandardScaler()

In [13]:
scaled_numeric_train_df = ss.fit_transform(numeric_train_df)
scaled_numeric_train_df = pd.DataFrame(scaled_numeric_train_df,index = numeric_train_df.index, columns=numeric_train_df.columns)

In [14]:
scaled_numeric_train_df

,ms_subclass,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod/add,mas_vnr_area,bsmtfin_sf_1,bsmtfin_sf_2,...,garage_area,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val,mo_sold,yr_sold
21,1.175949,-1.300412,-0.732817,-0.810443,-0.267261,-0.423530,-0.475964,-0.998392,0.612307,-0.196116,...,-1.083657,0.239319,-0.027640,-0.369465,-0.196116,-0.404063,0.0,0.0,1.329424,1.404879
81,-1.229402,-0.050326,0.273760,0.476731,-0.267261,0.921801,1.006847,1.373508,1.992157,-0.196116,...,3.025751,-1.374183,-0.005025,-0.369465,-0.196116,-0.404063,0.0,0.0,0.062561,1.404879
100,-1.229402,-0.593842,0.100676,0.476731,-0.267261,0.249135,0.183063,-0.059779,0.943397,-0.196116,...,-0.030121,0.254541,0.356806,-0.369465,-0.196116,-0.404063,0.0,0.0,0.907136,0.561951
138,-0.267261,-0.213380,0.328459,1.763906,-0.267261,0.585468,0.512576,1.100803,-1.529604,-0.196116,...,0.512050,0.178432,-0.593002,-0.369465,-0.196116,-0.404063,0.0,0.0,0.907136,-1.123903
151,-0.267261,1.145410,0.156744,0.476731,-0.267261,0.417302,0.512576,-0.338826,-1.085685,-0.196116,...,0.253287,0.254541,1.080470,-0.369465,-0.196116,-0.404063,0.0,0.0,0.062561,-1.123903
189,-0.267261,0.982355,-0.152959,-0.810443,-0.267261,0.585468,0.512576,0.479289,-1.529604,-0.196116,...,-0.054765,1.091735,0.899554,-0.369465,-0.196116,-0.404063,0.0,0.0,0.062561,-1.123903
190,-0.267261,-0.267732,0.153919,0.476731,-0.267261,0.417302,0.512576,0.904202,0.723287,-0.196116,...,-0.030121,0.010993,-0.525159,-0.369465,-0.196116,-0.404063,0.0,0.0,-1.626589,-1.123903
227,-0.267261,0.710597,-0.259103,0.476731,-0.267261,0.417302,0.677333,-0.221499,-1.529604,-0.196116,...,-0.116375,-0.278219,-0.479930,-0.369465,-0.196116,-0.404063,0.0,0.0,-0.359726,0.561951
246,2.138090,-1.083006,-0.763462,-2.097618,4.543441,-2.609693,-2.617801,-0.998392,-0.614020,5.099020,...,-1.052852,1.982206,-0.570388,-0.369465,-0.196116,-0.404063,0.0,0.0,-0.359726,-1.123903
317,-0.267261,-0.050326,-0.165456,0.476731,-0.267261,0.585468,0.512576,-0.414930,-1.150423,-0.196116,...,0.000685,0.498088,-0.479930,-0.369465,-0.196116,-0.404063,0.0,0.0,0.062561,-1.123903


In [15]:
#add id and PID back to numeric column
scaled_numeric_train_df = pd.concat([id_train_df,scaled_numeric_train_df], axis=1)
scaled_numeric_train_df

,id,pid,ms_subclass,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod/add,mas_vnr_area,...,garage_area,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val,mo_sold,yr_sold
21,351,527131030,1.175949,-1.300412,-0.732817,-0.810443,-0.267261,-0.423530,-0.475964,-0.998392,...,-1.083657,0.239319,-0.027640,-0.369465,-0.196116,-0.404063,0.0,0.0,1.329424,1.404879
81,367,527214050,-1.229402,-0.050326,0.273760,0.476731,-0.267261,0.921801,1.006847,1.373508,...,3.025751,-1.374183,-0.005025,-0.369465,-0.196116,-0.404063,0.0,0.0,0.062561,1.404879
100,1013,527214030,-1.229402,-0.593842,0.100676,0.476731,-0.267261,0.249135,0.183063,-0.059779,...,-0.030121,0.254541,0.356806,-0.369465,-0.196116,-0.404063,0.0,0.0,0.907136,0.561951
138,2331,527210040,-0.267261,-0.213380,0.328459,1.763906,-0.267261,0.585468,0.512576,1.100803,...,0.512050,0.178432,-0.593002,-0.369465,-0.196116,-0.404063,0.0,0.0,0.907136,-1.123903
151,2333,527212030,-0.267261,1.145410,0.156744,0.476731,-0.267261,0.417302,0.512576,-0.338826,...,0.253287,0.254541,1.080470,-0.369465,-0.196116,-0.404063,0.0,0.0,0.062561,-1.123903
189,2334,527212040,-0.267261,0.982355,-0.152959,-0.810443,-0.267261,0.585468,0.512576,0.479289,...,-0.054765,1.091735,0.899554,-0.369465,-0.196116,-0.404063,0.0,0.0,0.062561,-1.123903
190,2330,527210030,-0.267261,-0.267732,0.153919,0.476731,-0.267261,0.417302,0.512576,0.904202,...,-0.030121,0.010993,-0.525159,-0.369465,-0.196116,-0.404063,0.0,0.0,-1.626589,-1.123903
227,1012,527212050,-0.267261,0.710597,-0.259103,0.476731,-0.267261,0.417302,0.677333,-0.221499,...,-0.116375,-0.278219,-0.479930,-0.369465,-0.196116,-0.404063,0.0,0.0,-0.359726,0.561951
246,2328,527190050,2.138090,-1.083006,-0.763462,-2.097618,4.543441,-2.609693,-2.617801,-0.998392,...,-1.052852,1.982206,-0.570388,-0.369465,-0.196116,-0.404063,0.0,0.0,-0.359726,-1.123903
317,2332,527210050,-0.267261,-0.050326,-0.165456,0.476731,-0.267261,0.585468,0.512576,-0.414930,...,0.000685,0.498088,-0.479930,-0.369465,-0.196116,-0.404063,0.0,0.0,0.062561,-1.123903


# Categorical Features

#### Ordinal Features

In [16]:
ordinal_columns = ['utilities','land_slope','exter_qual',
       'bsmt_qual', 'bsmt_cond', 'bsmt_exposure','bsmtfin_type_1',
        'bsmtfin_type_2', 'heating_qc','electrical', 'kitchen_qual', 
        'functional','fireplace_qu', 'garage_finish', 'garage_qual',
        'garage_cond', 'paved_drive'] 

In [17]:
ordinal_train_df = categorical_train_df[ordinal_columns]
ordinal_train_df.head()

,utilities,land_slope,exter_qual,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_type_2,heating_qc,electrical,kitchen_qual,functional,fireplace_qu,garage_finish,garage_qual,garage_cond,paved_drive
21,AllPub,Gtl,Gd,Gd,TA,No,GLQ,Unf,Ex,SBrkr,Ex,Typ,Ex,Fin,TA,TA,Y
81,AllPub,Gtl,Ex,Ex,TA,No,GLQ,Unf,Ex,SBrkr,Ex,Typ,Gd,Fin,TA,TA,Y
100,AllPub,Gtl,Ex,Ex,TA,Gd,GLQ,Unf,Ex,SBrkr,Ex,Typ,Gd,Fin,TA,TA,Y
138,AllPub,Gtl,Ex,Ex,TA,Gd,Unf,Unf,Ex,SBrkr,Ex,Typ,Ex,Fin,TA,TA,Y
151,AllPub,Gtl,Gd,Ex,TA,Av,GLQ,Unf,Ex,SBrkr,Ex,Typ,Gd,Fin,TA,TA,Y


In [18]:
ordinal_null = ordinal_train_df.isna().sum().sort_values(ascending = False)
ordinal_null[ordinal_null > 0]

Series([], dtype: int64)

#### Ordinal categories that could mean 0 for NA

## Nominal Features

In [19]:
nominal_train_df = categorical_train_df.drop(columns = ordinal_columns)
nominal_train_df.head()

,ms_zoning,street,lot_shape,land_contour,lot_config,neighborhood,condition_1,condition_2,bldg_type,house_style,...,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,exter_cond,foundation,heating,central_air,garage_type,sale_type
21,RL,Pave,IR1,Lvl,Inside,StoneBr,Norm,Norm,TwnhsE,1Story,...,CompShg,CemntBd,CmentBd,None,TA,PConc,GasA,Y,Attchd,WD
81,RL,Pave,IR1,Lvl,CulDSac,StoneBr,Norm,Norm,1Fam,1Story,...,CompShg,VinylSd,VinylSd,Stone,TA,PConc,GasA,Y,Attchd,New
100,RL,Pave,IR1,HLS,CulDSac,StoneBr,Norm,Norm,1Fam,1Story,...,CompShg,VinylSd,VinylSd,BrkFace,TA,PConc,GasA,Y,Attchd,WD
138,RL,Pave,IR1,HLS,CulDSac,StoneBr,Norm,Norm,1Fam,2Story,...,CompShg,CemntBd,CmentBd,BrkFace,TA,PConc,GasA,Y,BuiltIn,New
151,RL,Pave,IR1,Lvl,Inside,StoneBr,Norm,Norm,1Fam,2Story,...,CompShg,CemntBd,CmentBd,Stone,TA,PConc,GasA,Y,BuiltIn,New


#### Nominal categories that could mean 0 for NA

## Encoding Ordinal Features

### Ordinal Dictionary

In [20]:
ordinal_col_list = ordinal_train_df.columns
print(ordinal_col_list)

Index(['utilities', 'land_slope', 'exter_qual', 'bsmt_qual', 'bsmt_cond',
       'bsmt_exposure', 'bsmtfin_type_1', 'bsmtfin_type_2', 'heating_qc',
       'electrical', 'kitchen_qual', 'functional', 'fireplace_qu',
       'garage_finish', 'garage_qual', 'garage_cond', 'paved_drive'],
      dtype='object')


In [21]:
ordinal_dict = [
    
#Utilities
{'AllPub':4,'NoSewr':3,'NoSeWa':2,'ELO':1},
#Land Slope
{'Gtl':3,'Mod':2,'Sev':1},
#Exter Qual
{'Ex':5,'Gd':4, 'TA':3, 'Fa':2,'Po':1},     
#Bsmt Qual
{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,0:0},
#Bsmt Cond
{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,0:0},
#Bsmt Exposure
{'Gd':4,'Av':3,'Mn':2,'No':1,0:0},
#BsmtFin Type 1
{'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,0:0},
#BsmtFinType 2
{'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,0:0},
#HeatingQC 
{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1},
#Electrical
{'SBrkr':5,'FuseA':4,'FuseF':3,'FuseP':2,'Mix':1},
#KitchenQual
{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1},
#Functional
{'Typ':8,'Min1':7,'Min2':6,'Mod':5,'Maj1':4,'Maj2':3,'Sev':2,'Sal':1},
#FireplaceQu
{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,0:0},
#Garage Finish
{'Fin':3,'RFn':2,'Unf':1,0:0},
#Garage Qual
{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,0:0},
#Garage Cond
{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,0:0},
#Paved Drive
{'Y':3,'P':2,'N':1},

]

In [22]:
#zip column names to dictionary values
named_ordinal_dict = dict(zip(ordinal_col_list,ordinal_dict))

In [23]:
def ordinal_replace(df,columns):
    for column in columns:
        df[column] = df[column].map(named_ordinal_dict[column])
    return df

In [24]:
#replace categorty value with values
encoded_ordinal_train_df = ordinal_replace(ordinal_train_df, ordinal_train_df.columns)
encoded_ordinal_train_df

,utilities,land_slope,exter_qual,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_type_2,heating_qc,electrical,kitchen_qual,functional,fireplace_qu,garage_finish,garage_qual,garage_cond,paved_drive
21,4,3,4,4,3,1,6,1,5,5,5,8,5,3,3,3,3
81,4,3,5,5,3,1,6,1,5,5,5,8,4,3,3,3,3
100,4,3,5,5,3,4,6,1,5,5,5,8,4,3,3,3,3
138,4,3,5,5,3,4,1,1,5,5,5,8,5,3,3,3,3
151,4,3,4,5,3,3,6,1,5,5,5,8,4,3,3,3,3
189,4,3,5,5,4,1,1,1,5,5,5,8,4,3,3,3,3
190,4,3,4,5,3,4,6,1,5,5,5,8,4,3,3,3,3
227,4,3,4,4,4,1,1,1,5,5,5,8,4,3,3,3,3
246,4,3,4,4,4,1,6,3,4,5,4,8,3,2,4,3,3
317,4,3,4,5,4,4,6,1,5,5,4,8,4,3,3,3,3


### Encoding Nominal Features

In [25]:
#all 0 values replaced from NA should be dropped when encoded.
dummies_nominal_train_df = pd.get_dummies(nominal_train_df, drop_first=True)

## Combine all dataframes back to one and save

In [26]:
#numerical, ordinal, and nominal data frames created earlier
dataframes = [scaled_numeric_train_df,encoded_ordinal_train_df,dummies_nominal_train_df]

In [27]:
#combine on index
cleaned_train_data = pd.concat(dataframes, axis=1)
cleaned_train_data.head()

,id,pid,ms_subclass,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod/add,mas_vnr_area,...,exterior_2nd_HdBoard,exterior_2nd_MetalSd,exterior_2nd_VinylSd,exterior_2nd_Wd Shng,mas_vnr_type_None,mas_vnr_type_Stone,exter_cond_Gd,exter_cond_TA,garage_type_BuiltIn,sale_type_WD
21,351,527131030,1.175949,-1.300412,-0.732817,-0.810443,-0.267261,-0.423530,-0.475964,-0.998392,...,0,0,0,0,1,0,0,1,0,1
81,367,527214050,-1.229402,-0.050326,0.273760,0.476731,-0.267261,0.921801,1.006847,1.373508,...,0,0,1,0,0,1,0,1,0,0
100,1013,527214030,-1.229402,-0.593842,0.100676,0.476731,-0.267261,0.249135,0.183063,-0.059779,...,0,0,1,0,0,0,0,1,0,1
138,2331,527210040,-0.267261,-0.213380,0.328459,1.763906,-0.267261,0.585468,0.512576,1.100803,...,0,0,0,0,0,0,0,1,1,0
151,2333,527212030,-0.267261,1.145410,0.156744,0.476731,-0.267261,0.417302,0.512576,-0.338826,...,0,0,0,0,0,1,0,1,1,0


In [28]:
#combine target back to cleaned dataframe on ID
final_train_df = pd.merge(cleaned_train_data,y, on = 'id')

In [31]:
final_train_df.to_csv('datasets/train_stonebrook_df.csv',index=False)